# Algorithmic Trading Using Deep Reinforcdment Learning

### Table of Contents

- [Introduction](#scrollTo=9SNR5Z82unXd)

- [Import Dependencies](#scrollTo=012Sf4GHumaL)

- [Data & Preprocessing](#scrollTo=fAq9RY7dtwTe)

- [Custom Trading Environment Setting](#scrollTo=z-g6RJHLpuKh)

- [Utility Functions](#scrollTo=jWElTzIctZ3E)

- [PPO Agent](#scrollTo=rz1CA85UkXPI)

  - [Agent setting](#scrollTo=bKxYFvx8nALx)

  - [Training](#scrollTo=VXm3OnAlnF51)

  - [Results and Validation](#scrollTo=3D6ypjL-nh8J)

- [DQN Agent](#scrollTo=F1SfonB9kXPR)

  - [Agent Setting](#scrollTo=hIqzHqzPn2Lv)

  - [Training](#scrollTo=6tBBq8TQn7JZ)

  - [Results and Validation](#scrollTo=SukTkoX8oExx)

- [Visualization](#scrollTo=V48fsZn7jv9Q)



## Introduction

In quantitative finance, stock trading is essentially a dynamic decision problem, that is, deciding where, at what price, and how much to trade in a highly stochastic, dynamic, and complex stock market. With recent advances in deep reinforcement learning (DRL) methods, sequential dynamic decision problems can be modeled and solved with a human-like approach.

<br>

In this poject, we examine the potential and performance of deep reinforcement learning to optimize stock trading strategies and thus maximize investment returns. Google stock is selected as our trading stock and the daily opening and closing price along with trading volume and several technical indicators are used as a training environment and trading market.

<br>

We present two trading agents based on deep reinforcement learning, one using Proximal Policy Otimization algorithm and the other based on Deep Q-Learing, to autonomously make trading decisions and generate returns in dynamic financial markets. The performance of these intelligent agents is compared with the performance of the buy and hold strategy. And at the end, it is shown that the proposed deep reinforcement learning approach performs better than the buy and hold benchmark in terms of risk assessment criteria and portfolio return.

---


**References:**
* Human-level control through deep reinforcement learning (Deep Q-Learning) : [paper](https://www.nature.com/articles/nature14236)
* Proximal Policy Optimization) : [paper](https://arxiv.org/abs/1707.06347), [blog](https://openai.com/blog/openai-baselines-ppo/), [spinning-up](https://spinningup.openai.com/en/latest/algorithms/ppo.html)

## Import Dependencies

In [ ]:
%%capture
!pip install talib-binary
!pip install gym_anytrading
!pip install quantstats
!pip install stable_baselines3
!pip install pyfolio
!pip install --upgrade gym==0.25.2
!pip install stable_baselines

In [ ]:
import os
import math
import talib
import numpy as np
import pandas as pd
from scipy.stats import t
from pandas_datareader import data as web
import pandas_datareader as pdr
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from gym.utils import seeding
import gym
from gym import spaces
import logging
import datetime
import pyfolio.timeseries as ts
import scipy.stats as st

from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
# from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
import matplotlib.pyplot as plt
import seaborn as sns
import quantstats as qs

from stable_baselines3 import A2C, DDPG, DQN, PPO, TD3, SAC
# from stable_baselines import TRPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# import torch
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

import torch as th
from torch import nn

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy

from sklearn.preprocessing import scale


DECIMAL_SIGNS = 5
rnd = lambda x: round(x, DECIMAL_SIGNS)

#===========